## Setup i konfiguracja

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 3 - Integracje i Governance
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Klaster ML (Databricks Runtime ML 13.0+)
  - Dostęp do Unity Catalog (dla Feature Store i Vector Search)
  - (Opcjonalnie) Skonfigurowany Vector Search Endpoint
  - (Opcjonalnie) Dostęp do Model Serving (Foundation Models)

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie jak Databricks wspiera pełny cykl życia AI/ML.

**Podstawowe pojęcia:**
- **MLflow**: Platforma open-source do zarządzania cyklem życia ML (eksperymenty, modele, wdrożenia).
- **Feature Store**: Scentralizowane repozytorium cech, zapewniające spójność między treningiem a inferencją.
- **Vector Search**: Baza danych wektorowych zintegrowana z Delta Lake, kluczowa dla RAG (Retrieval Augmented Generation).

## Izolacja per użytkownik

In [ ]:
%run ../00_setup

## Konfiguracja środowiska

In [ ]:
import mlflow
import pandas as pd
from pyspark.sql import functions as F
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from databricks import feature_store
from databricks.feature_store import FeatureStoreClient

# Ustawienie katalogu i schematu
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {GOLD_SCHEMA}")

print(f"Working in: {CATALOG}.{GOLD_SCHEMA}")

## Sekcja 1: MLflow - Tracking & Registry

Zbudujemy prosty model przewidujący wartość zamówienia i zarejestrujemy go w MLflow.

In [ ]:
# 1. Konfiguracja eksperymentu
username = spark.sql("SELECT current_user()").collect()[0][0]
experiment_path = f"/Users/{username}/KION_ML_Experiment"

mlflow.set_experiment(experiment_path)
print(f"Eksperyment: {experiment_path}")

In [ ]:
# 2. Przygotowanie danych
df_sales = spark.sql(f"""
    SELECT 
        total_amount as label,
        quantity,
        month(order_date) as month,
        year(order_date) as year
    FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales
    WHERE total_amount IS NOT NULL
""").toPandas()

X = df_sales.drop("label", axis=1)
y = df_sales["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Train shape: {X_train.shape}")

In [ ]:
# 3. Trening i logowanie
with mlflow.start_run(run_name="RandomForest_v1") as run:
    # Parametry
    n_estimators = 50
    mlflow.log_param("n_estimators", n_estimators)
    
    # Model
    model = RandomForestRegressor(n_estimators=n_estimators)
    model.fit(X_train, y_train)
    
    # Metryki
    mse = mean_squared_error(y_test, model.predict(X_test))
    mlflow.log_metric("mse", mse)
    
    # Logowanie modelu
    mlflow.sklearn.log_model(model, "model")
    
    print(f"Run ID: {run.info.run_id}, MSE: {mse:.2f}")

## Sekcja 2: Feature Store

Feature Store pozwala na definiowanie cech raz i używanie ich wszędzie.

In [ ]:
# Definicja tabeli cech (Feature Table)
fs = FeatureStoreClient()
feature_table_name = f"{CATALOG}.{GOLD_SCHEMA}.customer_features_{username.split('@')[0].replace('.', '_')}"

# Logika obliczania cech
features_df = spark.sql(f"""
    SELECT 
        customer_id,
        count(*) as order_count,
        sum(total_amount) as total_spend,
        avg(total_amount) as avg_order_value
    FROM {CATALOG}.{GOLD_SCHEMA}.fact_sales
    GROUP BY customer_id
""")

# Zapis do Feature Store
# Uwaga: create_table wymaga unikalnej nazwy i Primary Key
try:
    fs.create_table(
        name=feature_table_name,
        primary_keys=["customer_id"],
        df=features_df,
        description="Podstawowe cechy klienta: liczba zamówień, wydatki."
    )
    print(f"Tabela cech utworzona: {feature_table_name}")
except Exception as e:
    print(f"Tabela prawdopodobnie istnieje lub błąd uprawnień: {e}")
    # Jeśli istnieje, możemy ją zaktualizować:
    # fs.write_table(name=feature_table_name, df=features_df, mode="merge")

## Sekcja 3: Vector Search (RAG Foundation)

Vector Search to klucz do budowania systemów RAG. Pozwala wyszukiwać dokumenty podobne semantycznie do zapytania użytkownika.

**Wymagania:**
1.  **Vector Search Endpoint** (musi być utworzony w Compute -> Vector Search).
2.  **Tabela źródłowa** z włączonym Change Data Feed.

In [ ]:
# 1. Przygotowanie danych tekstowych (Symulacja bazy wiedzy)
# Tworzymy tabelę z opisami produktów (w realnym scenariuszu to byłyby PDFy, dokumentacja itp.)

docs_df = spark.createDataFrame([
    (1, "Wózek widłowy elektryczny, udźwig 2 tony, do pracy wewnątrz magazynu."),
    (2, "Wózek spalinowy Diesel, udźwig 5 ton, do pracy na zewnątrz, trudny teren."),
    (3, "Ręczny wózek paletowy, udźwig 500kg, do lekkich prac."),
    (4, "System automatyzacji magazynu, regały wysokiego składowania.")
], ["id", "text"])

source_table = f"{CATALOG}.{GOLD_SCHEMA}.product_docs_rag"
docs_df.write.format("delta").mode("overwrite").option("delta.enableChangeDataFeed", "true").saveAsTable(source_table)
print(f"Tabela źródłowa utworzona: {source_table}")

In [ ]:
# 2. Konfiguracja Vector Search (Kod właściwy)
# Uwaga: To wymaga aktywnego Endpointu Vector Search. 
# Jeśli go nie ma, kod rzuci błąd, ale pokazuje poprawną ścieżkę implementacji.

from databricks.vector_search.client import VectorSearchClient

# Nazwa endpointu (musi istnieć w UI)
VECTOR_SEARCH_ENDPOINT_NAME = "kion_vector_endpoint_demo" 
index_name = f"{CATALOG}.{GOLD_SCHEMA}.product_docs_index"

try:
    vsc = VectorSearchClient()
    
    # Sprawdź czy endpoint istnieje
    endpoints = vsc.list_endpoints()
    endpoint_exists = any(e['name'] == VECTOR_SEARCH_ENDPOINT_NAME for e in endpoints.get('endpoints', []))
    
    if endpoint_exists:
        print(f"Endpoint {VECTOR_SEARCH_ENDPOINT_NAME} znaleziony. Tworzenie indeksu...")
        
        # Tworzenie indeksu (Delta Sync Index)
        # To automatycznie zarządza embeddingami (używając modelu Databricks BGE lub OpenAI)
        vsc.create_delta_sync_index(
            endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
            source_table_name=source_table,
            index_name=index_name,
            pipeline_type="TRIGGERED",
            primary_key="id",
            embedding_source_column="text",
            embedding_model_endpoint_name="databricks-bge-large-en" # Model wbudowany
        )
        print("Indeks jest tworzony w tle...")
    else:
        print(f"WARN: Endpoint '{VECTOR_SEARCH_ENDPOINT_NAME}' nie istnieje. Utwórz go w UI (Compute -> Vector Search).")
        print("Poniżej symulacja działania (dla celów demo).")

except Exception as e:
    print(f"Błąd Vector Search (czy biblioteka jest zainstalowana?): {e}")

In [ ]:
# 3. Symulacja wyszukiwania (gdyby Vector Search działał)
# query = "Szukam wózka do ciężkich prac na dworze"
# results = vsc.get_index(endpoint_name, index_name).similarity_search(
#     query_text=query,
#     columns=["id", "text"],
#     num_results=1
# )
# print(results)

# --- SYMULACJA LOKALNA (Dla celów edukacyjnych, gdy brak infrastruktury) ---
print("\n--- SYMULACJA WYNIKÓW ---")
print("Query: 'Szukam wózka do ciężkich prac na dworze'")
print("Najbardziej podobny dokument:")
print("ID: 2")
print("Text: Wózek spalinowy Diesel, udźwig 5 ton, do pracy na zewnątrz, trudny teren.")
print("Score: 0.89")

## Sekcja 4: GenAI & Model Serving

Integracja z modelami LLM (Large Language Models) poprzez Databricks Model Serving.

In [ ]:
# Wywołanie modelu LLM (np. DBRX, Llama 3)
import mlflow.deployments

# Lista dostępnych endpointów (może wymagać uprawnień)
# client = mlflow.deployments.get_deploy_client("databricks")
# endpoints = client.list_endpoints()

def query_llm(prompt):
    try:
        client = mlflow.deployments.get_deploy_client("databricks")
        
        # Używamy modelu 'databricks-dbrx-instruct' lub 'databricks-meta-llama-3-70b-instruct'
        # Te modele są często dostępne jako "Pay-per-token"
        response = client.predict(
            endpoint="databricks-dbrx-instruct",
            inputs={
                "messages": [
                    {"role": "system", "content": "Jesteś ekspertem od wózków widłowych KION."},
                    {"role": "user", "content": prompt}
                ],
                "max_tokens": 100
            }
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        return f"Nie udało się połączyć z modelem: {e}"

# Test
prompt = "Jakie są zalety wózków elektrycznych?"
print(f"Pytanie: {prompt}")
print(f"Odpowiedź LLM: \n{query_llm(prompt)}")

## Podsumowanie

1.  Zbudowaliśmy pipeline treningowy w **MLflow**.
2.  Utworzyliśmy tabelę cech w **Feature Store**.
3.  Pokazaliśmy jak skonfigurować **Vector Search** dla RAG.
4.  Skorzystaliśmy z **GenAI** do generowania odpowiedzi.

## Czyszczenie zasobów

In [ ]:
# spark.sql(f"DROP TABLE IF EXISTS {feature_table_name}")
# spark.sql(f"DROP TABLE IF EXISTS {source_table}")
print("Zasoby zachowane.")